# Laboratorio 6

- Irving Acosta 22781
- Ricardo Chuy 221007

[Repositorio](https://github.com/wwIrvingww/Lab6_DS.git)

## Avances 
### Carga de datos

### Librerias

In [ ]:
from pathlib import Path
import json
import pandas as pd

In [ ]:


# -------- helpers --------
def get_id_str(d, base="id"):
    """Devuelve d[f"{base}_str"] si existe; si no, usa d[base] -> str."""
    if not isinstance(d, dict):
        return None
    if f"{base}_str" in d and d[f"{base}_str"] is not None:
        return str(d[f"{base}_str"])
    v = d.get(base)
    if v is None:
        return None
    try:
        if isinstance(v, float) and v.is_integer():
            return str(int(v))
    except Exception:
        pass
    return str(v)

def flatten_tweet(t):
    u = t.get("user", {}) if isinstance(t.get("user"), dict) else {}

    # mentionedUsers
    musers = t.get("mentionedUsers", [])
    if not isinstance(musers, list):
        musers = []
    mentioned_ids = []
    mentioned_usernames = []
    for mu in musers:
        if isinstance(mu, dict):
            mid = get_id_str(mu, "id")
            if mid: mentioned_ids.append(mid)
            if mu.get("username"): mentioned_usernames.append(mu["username"])

    # quotedTweet
    qt = t.get("quotedTweet", {}) if isinstance(t.get("quotedTweet"), dict) else None
    qt_user = qt.get("user", {}) if qt and isinstance(qt.get("user"), dict) else {}

    # media counts
    media = t.get("media", {}) if isinstance(t.get("media"), dict) else {}
    photos_cnt = len(media.get("photos", [])) if isinstance(media.get("photos"), list) else 0
    videos_cnt = len(media.get("videos", [])) if isinstance(media.get("videos"), list) else 0
    animated_cnt = len(media.get("animated", [])) if isinstance(media.get("animated"), list) else 0

    hashtags = t.get("hashtags", []) or []
    cashtags = t.get("cashtags", []) or []

    # inReplyToUser
    iru = t.get("inReplyToUser")
    inReplyToUser_val = iru.get("username") if isinstance(iru, dict) else iru

    # descriptionLinks y pinnedIds del user
    user_descriptionLinks_cnt = len(u.get("descriptionLinks", [])) if isinstance(u.get("descriptionLinks"), list) else 0
    user_pinnedIds_val = u.get("pinnedIds")
    if isinstance(user_pinnedIds_val, list):
        user_pinnedIds_val = ",".join(str(x) for x in user_pinnedIds_val)

    return {
        "id": get_id_str(t, "id"),
        "date": t.get("date"),

        "user": u.get("username"),
        "user_id": get_id_str(u, "id"),
        "user_displayname": u.get("displayname"),
        "user_rawDescription": u.get("rawDescription"),
        "user_created": u.get("created"),
        "user_followersCount": u.get("followersCount"),
        "user_friendsCount": u.get("friendsCount"),
        "user_statusesCount": u.get("statusesCount"),
        "user_favouritesCount": u.get("favouritesCount"),
        "user_listedCount": u.get("listedCount"),
        "user_mediaCount": u.get("mediaCount"),
        "user_location": u.get("location"),
        "user_protected": u.get("protected"),
        "user_verified": u.get("verified"),
        "user_blue": u.get("blue"),
        "user_blueType": u.get("blueType"),
        "user_descriptionLinks": user_descriptionLinks_cnt,
        "user_pinnedIds": user_pinnedIds_val,

        "lang": t.get("lang"),
        "rawContent": t.get("rawContent"),
        "replyCount": t.get("replyCount"),
        "retweetCount": t.get("retweetCount"),
        "likeCount": t.get("likeCount"),
        "quoteCount": t.get("quoteCount"),
        "bookmarkedCount": t.get("bookmarkedCount"),
        "conversationId": get_id_str(t, "conversationId"),

        "hashtags": ",".join(hashtags) if hashtags else None,
        "cashtags": ",".join(cashtags) if cashtags else None,
        "mentionedUsers_ids": ",".join(mentioned_ids) if mentioned_ids else None,
        "mentionedUsers_usernames": ",".join(mentioned_usernames) if mentioned_usernames else None,

        "viewCount": t.get("viewCount"),

        "quotedTweet_id": get_id_str(qt, "id") if qt else None,
        "quotedTweet_date": qt.get("date") if qt else None,
        "quotedTweet_user_id": get_id_str(qt_user, "id") if qt_user else None,
        "quotedTweet_user_username": qt_user.get("username") if qt_user else None,

        "place": json.dumps(t.get("place"), ensure_ascii=False) if isinstance(t.get("place"), (dict, list)) else t.get("place"),
        "inReplyToTweetId": get_id_str(t, "inReplyToTweetId"),
        "inReplyToUser": inReplyToUser_val,
        "sourceLabel": t.get("sourceLabel"),

        "media_photos_count": photos_cnt,
        "media_videos_count": videos_cnt,
        "media_animated_count": animated_cnt,
    }

# -------- lectura --------
src = Path("traficogt.txt")
rows = []
bad_lines = 0
with src.open("r", encoding="utf-16") as f:
    for i, ln in enumerate(f, 1):
        ln = ln.strip()
        if not ln:
            continue
        try:
            t = json.loads(ln)
            rows.append(flatten_tweet(t))
        except json.JSONDecodeError:
            bad_lines += 1

print(f"Filas parseadas: {len(rows)} | líneas malas: {bad_lines}")

df = pd.DataFrame(rows)

# Reordenar columnas
ordered_cols = [
    "id","date","user",
    "user_id","user_displayname","user_rawDescription","user_created",
    "user_followersCount","user_friendsCount","user_statusesCount","user_favouritesCount",
    "user_listedCount","user_mediaCount","user_location","user_protected","user_verified",
    "user_blue","user_blueType","user_descriptionLinks","user_pinnedIds",
    "lang","rawContent","replyCount","retweetCount","likeCount","quoteCount",
    "bookmarkedCount","conversationId",
    "hashtags","cashtags",
    "mentionedUsers_ids","mentionedUsers_usernames",
    "viewCount",
    "quotedTweet_id","quotedTweet_date","quotedTweet_user_id","quotedTweet_user_username",
    "place","inReplyToTweetId","inReplyToUser","sourceLabel",
    "media_photos_count","media_videos_count","media_animated_count",
]
df = df.reindex(columns=[c for c in ordered_cols if c in df.columns])

print("Dimensiones:", df.shape)
display(df.head(3))

# Guardar CSV en UTF-8
df.to_csv("traficogt_flat.csv", index=False, encoding="utf-8")
print("CSV escrito en: traficogt_flat.csv")


Filas parseadas: 5604 | líneas malas: 1
Dimensiones: (5604, 44)


,id,date,user,user_id,user_displayname,user_rawDescription,user_created,user_followersCount,user_friendsCount,user_statusesCount,...,quotedTweet_date,quotedTweet_user_id,quotedTweet_user_username,place,inReplyToTweetId,inReplyToUser,sourceLabel,media_photos_count,media_videos_count,media_animated_count
0,1834236045598056867,2024-09-12 14:22:06+00:00,traficogt,93938886,traficoGT,Noticias de ciudad de Guatemala,2009-12-01 20:42:19+00:00,314368,137,52385,...,2024-09-12 00:39:56+00:00,976875408,monymmorales,None,None,None,Twitter for Android,0,0,0
1,1834029142565658846,2024-09-12 00:39:56+00:00,monymmorales,976875408,Mony,Iglesia y estado son asunto separado.\nCatólic...,2012-11-28 20:16:36+00:00,5502,4887,180436,...,None,None,None,None,None,None,Twitter for Android,0,0,0
2,1834039491826180424,2024-09-12 01:21:04+00:00,animaldgalaccia,1730828822029750272,Jairo De La Nada,Sancarlista. Estudiante de filosofía. Ideológi...,2023-12-02 05:58:56+00:00,571,158,8875,...,None,None,None,None,None,PNCdeGuatemala,Twitter for iPhone,0,0,0


CSV escrito en: traficogt_flat.csv
